In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=2d8d025d0a307ac958e29354175c827173d1477cb8426b24b0287844b57f9a43
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 4.5MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F2 '''
def F2(X):
    f = bn.F2()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 3336.11 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 4.96 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_2_1000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_2_200Samples.csv').iloc[:,-1]
path = "train_latent_50D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_50D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,26):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 25000)

n_obs , dim =  30, 25
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,75])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=822996, Mon Apr 13 13:03:28 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 5.323192357275825e+07 1.0e+00 1.19e-01  1e-01  1e-01 0:00.0
    2     26 5.247972445265356e+07 1.1e+00 1.18e-01  1e-01  1e-01 0:00.0
    3     39 4.945409568566605e+07 1.1e+00 1.22e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [54605019.96264509] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [54099329.23387733] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 -3.035838109203137e+07 1.8e+00 6.85e-03  6e-03  7e-03 0:01.1
  200   2600 -3.045560803979741e+07 2.2e+00 1.39e-04  1e-04  1e-04 0:02.2
  300   3900 -3.045563403391053e+07 2.4e+00 3.06e-06  2e-06  2e-06 0:03.2
  400   5200 -3.045563404214177e+07 2.7e+00 3.16e-08  2e-08  2e-08 0:04.3
  500   6500 -3.045563404214218e+07 3.1e+00 3.47e-09  2e-09  2e-09 0:05.4
  600   7800 -3.045563404214218e+07 3.8e+00 1.26e-09  5e-10  8e-10 0:06.4
  700   9100 -3.045563404214217e+07 4.6e+00 8.87e-10  3e-10  6e-10 0:07.5
  800  10400 -3.045563404214220e+07 5.7e+00 1.96e-09  7e-10  1e-09 0:08.6
  900  11700 -3.045563404214218e+07 6.1e+00 1.90e-09  6e-10  1e-09 0:09.7
 1000  13000 -3.045563404214220e+07 7.1e+00 1.60e-09  6e-10  1e-09 0:10.7
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  14300 -3.045563404214220e+07 8.5e+00 4.98e-10  2e-10  3e-10 0:11.8
 1200  15600 -3.045563404214220e+07 1.0e+01 4.81e-10  2e-10  3e-10 0:12.8
 1300  16900 -3.045563404214218e+07 1.1e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [52123863.97414719] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [52123883.42584834] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 5.211057512037780e+07 1.8e+00 5.64e-03  4e-03  5e-03 0:00.9
  200   2600 5.211056443392910e+07 1.8e+00 6.18e-05  4e-05  5e-05 0:01.7
  300   3900 5.211056443276653e+07 2.0e+00 1.09e-06  6e-07  8e-07 0:02.6


...

  400   5200 5.211056443276630e+07 2.0e+00 6.56e-08  3e-08  4e-08 0:03.6


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=52110564.432766, sigma=7.43e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=394)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=52110564.432766, sigma=7.29e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=396)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=52110564.432766, sigma=5.76e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=402)
  ')')
/usr/local/li

  433   5629 5.211056443276630e+07 2.3e+00 4.28e-08  2e-08  3e-08 0:03.9
termination on tolfun=1e-11 (Mon Apr 13 13:16:33 2020)
termination on tolfunhist=1e-12 (Mon Apr 13 13:16:33 2020)
termination on flat fitness=please (re)consider how to compute the fitness more elaborately if sigma=4.28e-08 is large (Mon Apr 13 13:16:33 2020)
final/bestever f-value = 5.211056e+07 5.211056e+07
incumbent solution: [-0.30613309 -0.023316   -0.14223393 -0.03540581 -0.25712186 -0.02472482
  0.25683028  0.05573409 ...]
std deviations: [2.22154723e-08 2.27807292e-08 2.34846449e-08 2.44295485e-08
 2.18718694e-08 2.30848294e-08 2.72992478e-08 2.23591971e-08 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=936670, Mon Apr 13 13:16:33 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 9.019343269264299e+07 1.0e+00 1.22e-01  1e-01  1e-01 0:00.0
    2     26 8.358198334183513e+07 1.1e+00 1.21e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=52110564.432766, sigma=4.53e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=429)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=52110564.432766, sigma=4.43e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=430)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=52110564.432766, sigma=4.37e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=431)
  ')')
/usr/local/li

    3     39 6.620117281981316e+07 1.1e+00 1.25e-01  1e-01  1e-01 0:00.0
   77   1001 -7.650299832875203e+13 1.8e+00 1.24e+02  1e+02  1e+02 0:01.1
termination on tolfacupx=1000.0 (Mon Apr 13 13:16:34 2020)
final/bestever f-value = -6.610520e+13 -7.650300e+13
incumbent solution: [ 353.7954761   -17.25271046 -261.40992514  -64.54283875  151.17205016
 -406.06071126 -174.67673316 -205.30430188 ...]
std deviations: [116.8631232  113.21432345 115.83469262 114.53009482 109.48688352
 119.36290035 110.3173567  118.53643261 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F2_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F2(X_Values.iloc[i,:25])
    SVM_Fun [i] = F2(X_Values.iloc[i,25:50])
    ELN_Fun [i] = F2(X_Values.iloc[i,50:75])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([12886571.09559132]), count=array([1])) 0.02502181832175318
SVM
ModeResult(mode=array([12581199.0105495]), count=array([1])) 0.5896651231886347
ELN
ModeResult(mode=array([2.89476054e+10]), count=array([1])) 166276513371.09732
